In [0]:
!pip install lightgbm

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
worksheet = gc.open('data_nlp').sheet1

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
rows = worksheet.get_all_values()
print(rows)
nlp = pd.DataFrame.from_records(rows)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
header = nlp.iloc[0]
nlp = nlp[1:]
nlp.columns = header
nlp.head()

,id,price,brand,description,categories,sub_category
1,eac7efa5dbd3d667f26eb3d3ab504464,4,Hornby,Product Description Hornby 2014 Catalogue Box ...,Toys,Model Trains & Railway Sets
2,b17540ef7e86e461d37f3ae58b7b72ac,21,FunkyBuys,Size Name:Large FunkyBuys® Large Christmas Hol...,Toys,Model Trains & Railway Sets
3,348f344247b0c1a935b1223072ef9d8a,12,ccf,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Toys,Model Trains & Railway Sets
4,e12b92dbb8eaee78b22965d2a9bbbd9f,49,Hornby,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Toys,Model Trains & Railway Sets
5,e33a9adeed5f36840ccc227db4682a36,40,Hornby,Product Description Hornby RailRoad 0-4-0 Gild...,Toys,Model Trains & Railway Sets


In [0]:
nlp['concat'] = nlp['brand'] + " " + nlp['description'] + " " +nlp['categories'] + " " + nlp['sub_category']

In [0]:
X = nlp['concat']
y = nlp['price']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
seed = 101

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn import metrics

def tokenizer(text):
    return re.findall(r'[a-z0-9]+', text.lower())

vect = CountVectorizer(tokenizer=tokenizer)
regressor = LinearRegression()  


In [0]:
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [0]:
regressor.fit(X_train_vect, y_train)
y_pred = regressor.predict(X_test_vect)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 262.24980644765856


In [0]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators=400, max_depth= 5, min_samples_split= 2,
                                         learning_rate= 0.1, loss= 'ls')

In [0]:
clf.fit(X_train_vect, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=400,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
y_pred = clf.predict(X_test_vect)

In [0]:
clf.score(X_test_vect, y_test)

0.9180603233132953

In [0]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 64.17906475088436


In [0]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, learning_curve, ShuffleSplit, RandomizedSearchCV

In [0]:
lgb = LGBMRegressor(objective = 'regression_l1',
                    subsample = 0.1,
                     random_state = 0,
                     num_leaves = 120,
                     n_estimators = 1000,
                     min_split_gain = 0.06999,
                     min_data_in_leaf = 12,
                     max_depth = 56,
                     learning_rate = 0.005,
                     colsample_bytree = 0.4,
                     boosting_type = 'gbdt')

In [0]:
lgb.fit(X_train_vect, y_train)
#preds = lgb.predict(X_test_vect)